## Библиотеки

In [6]:
from sshtunnel import SSHTunnelForwarder
from pymongo import MongoClient
from start import start_parser
import openpyxl as px # работа с  xlsx файлами
import codecs

## Подключение к бд

In [7]:
# Боевая бд
"""MONGO_PORT = 55556
MONGO_DB = "fincase"
MONGO_USER = "shesterin"
MONGO_PASS = "sK349Acm9qwD92s"
SERVER_HOST = '192.168.0.78'"""

# Тестовая бд
MONGO_PORT = 55556
MONGO_DB = "fincase"
MONGO_USER = "root"
MONGO_PASS = "Gfcbyu&478532@#Gfhcbyu"
SERVER_HOST = '185.98.83.65'
SERVER_PORT = 20032

server = SSHTunnelForwarder(
    (SERVER_HOST, SERVER_PORT),
    ssh_username = MONGO_USER,
    ssh_password = MONGO_PASS,
    remote_bind_address = ('localhost', MONGO_PORT)
)

server.start()
print("Connection established with local bin port: ", server.local_bind_port)

db = MongoClient('localhost', server.local_bind_port)[MONGO_DB]

Connection established with local bin port:  36169


2019-10-18 15:58:44,167| ERROR   | Could not establish connection from ('127.0.0.1', 36169) to remote side of the tunnel


## Основная функция

In [8]:
def get_urls(db):
    try:
        collection_name = "building"
        print (collection_name + ' start!')
        for a in db[collection_name].find({"$and":
                                                    [{"pageId": {"$ne": None}}, 
                                                     {"sematic":{"$eq": None}}]}, 
                                                     {"building_house_line_type": 1, 
                                                      "condition_type": 1, 
                                                      "description": 1,
                                                      "vat_type" : 1, 
                                                      "pageId": 1}).batch_size(30):
            data = a['description']
            result = start_parser(data)   
            if a['building_house_line_type'] == None:
                db[collection_name].update_one({"pageId": a["pageId"]},
                                                        {"$set": 
                                                        {"building_house_line_type": result.iloc[0]["houselinetype"]}})
            if a['condition_type'] == None:
                db[collection_name].update_one({"pageId": a["pageId"]},
                                                        {"$set": 
                                                        {"condition_type": result.iloc[0]["conditiontype"]}})
            if a['vat_type'] == None:
                db[collection_name].update_one({"pageId": a["pageId"]},
                                                        {"$set": 
                                                        {"vat_type": result.iloc[0]["vattype"]}})           
            db[collection_name].update_one({"pageId": a["pageId"]},
                                                    {"$set": 
                                                    {"sematic": "true"}})
        print (collection_name + ' done!')
        
    except Exception as e:
        print(e)

def save_sematic_info(collection, id):
    collection.update_one({"pageId": id}, {"$set": {" sematic": 'true'}})    

## Запуск 

In [10]:
get_urls(db)

building start!


KeyboardInterrupt: 

In [9]:
array_building = [0]*3
i=0
for a in db["building"].find():
    
    i+=1
    
    if a["building_house_line_type"] == None:
        array_building[0] += 1
                        
    if a['condition_type'] == None:
        array_building[1] += 1
        
    if a['vat_type'] == None:
        array_building[2] += 1
print(i)

print ("Table buildings:" + str(array_building))

21623
Table buildings:[0, 0, 0]


## Выгружаем данные в csv формат

In [10]:
request = db["building"].find({"$and":
                                            [{"pageId": {"$ne": None}}, 
                                             {"sematic":{"$eq": "true"}}]}, 
                                             {"building_house_line_type": 1, 
                                              "condition_type": 1, 
                                              "description": 1,
                                              "vat_type" : 1, 
                                              "pageId": 1}).batch_size(30)
with codecs.open("building_mongo.csv", mode='w', encoding='utf-8') as f:
    f.flush()
    j = 0
    keys = []
    for line in request:
        j += 1
        if j == 1:
            keys = line.keys()
            f.write("\t".join(str(i) for i in keys))
            f.write("\n")
        if j % 10 == 0:
            for k in keys:
                f.write(str(line[k]).strip().replace("\n", " "))
                f.write("\t")
            f.write("\n")
    f.flush()

In [37]:
file = 'building_mongo.xlsx'
work_sheet = u'Лист1'
wb = px.load_workbook(file, data_only=True)
sheet = wb[work_sheet]

request = client[MONGO_DB]["building"].find({"$and":
                                                    [{"pageId": {"$ne": None}}, 
                                                     {"sematic":{"$eq": "true"}}]}, 
                                                     {"building_house_line_type": 1, 
                                                      "condition_type": 1, 
                                                      "description": 1,
                                                      "vat_type" : 1, 
                                                      "pageId": 1}).batch_size(30)
j = 0
for line in request:
    j += 1
    if j == 1:
        k = 0
        for note in line.keys():
            k += 1
            sheet.cell(row = j, column = k).value = str(note)
    if j % 10 == 0:
        k = 0
        for note in line.values():
            k += 1
            sheet.cell(row = j/10+1, column = k).value = str(note)
wb.save("building_mongo.xlsx") 

2019-10-16 18:14:56,179| ERROR   | Could not establish connection from ('127.0.0.1', 45371) to remote side of the tunnel
2019-10-16 18:14:56,184| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:15:06,194| ERROR   | Could not establish connection from ('127.0.0.1', 45371) to remote side of the tunnel
2019-10-16 18:15:06,199| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:15:16,260| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:15:26,282| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:15:36,299| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:15:46,317| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:15:56,339| ERROR  

2019-10-16 18:25:57,341| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:26:07,358| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:26:17,375| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:26:27,390| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:26:37,406| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:26:47,423| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:26:57,439| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:27:07,458| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:27:17,475| ERROR  

2019-10-16 18:37:18,454| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:37:28,472| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:37:38,489| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:37:48,506| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:37:58,522| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:38:08,539| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:38:18,556| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:38:28,573| ERROR   | Could not establish connection from ('127.0.0.1', 45653) to remote side of the tunnel
2019-10-16 18:38:38,592| ERROR  